In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms
import datetime
import time
import torch
import torch.nn as nn
import mat73
from pathlib import Path
import os
from torchsummary import summary

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float32)

In [2]:
path_dataset_folder = Path('./data/[1] simple linear')

path_file_input = path_dataset_folder / 'xTrain.mat'
path_file_output = path_dataset_folder / 'yTrain.mat'
path_train_data = (path_file_input, path_file_output)
torch.set_default_dtype(torch.float32)

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, path_train_data):
        self.input_data = mat73.loadmat(path_train_data[0])
        self.output_data = mat73.loadmat(path_train_data[1])
        # input_x = input_data['input'][:,0]
        # input_t = input_data['input'][:,1]
        # output_u = output_data['output']

    def __len__(self):
        return len(self.input_data['input'])

    def __getitem__(self, i):
        # input_data_sample = torch.from_numpy(self.input_data['input'][i, :], dtype=torch.float32)
        input_data_x = torch.from_numpy(np.asarray(self.input_data['input'][i, 0].astype('float32'))).unsqueeze(0)
        input_data_t = torch.from_numpy(np.asarray(self.input_data['input'][i, 1].astype('float32'))).unsqueeze(0)
        # output_data_sample = torch.from_numpy(self.output_data['output'][i], dtype=torch.float32)
        output_data_sample = torch.from_numpy(np.asarray(self.output_data['output'][i].astype('float32')))
        return input_data_x, input_data_t, output_data_sample

dataset_simple = MyDataset(path_train_data)
my_batch_size = int(np.power(2, 12))
train_dataloader=torch.utils.data.DataLoader(dataset_simple, batch_size=my_batch_size, shuffle=True)

In [53]:
for i, (train_input_x, train_input_t, train_output) in tqdm(enumerate(train_dataloader)):
    # print(train_output)
    print(train_output.size())

In [81]:
train_input_x = train_input_x.to(device)
train_input_t = train_input_t.to(device)
train_output = train_output.to(device)

train_input_x.requires_grad = True
train_input_t.requires_grad = True


NameError: name 'train_input_x' is not defined

In [3]:
class MyBenchModel(nn.Module):
    def __init__(self):
        super(MyBenchModel, self).__init__()
        #You code after this line

        self.my_layer = nn.Sequential(nn.Linear(2, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 1))

    def forward(self, x, t):
        #You code after this line
        x = torch.cat((x, t), dim=1)
        x = self.my_layer(x)
        return x

model = MyBenchModel().to(device)

In [4]:
model

MyBenchModel(
  (my_layer): Sequential(
    (0): Linear(in_features=2, out_features=60, bias=True)
    (1): Tanh()
    (2): Linear(in_features=60, out_features=60, bias=True)
    (3): Tanh()
    (4): Linear(in_features=60, out_features=60, bias=True)
    (5): Tanh()
    (6): Linear(in_features=60, out_features=60, bias=True)
    (7): Tanh()
    (8): Linear(in_features=60, out_features=60, bias=True)
    (9): Tanh()
    (10): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [102]:
summary(model, input_size=([(1,), (1,)]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 60]             180
              Tanh-2                   [-1, 60]               0
            Linear-3                   [-1, 60]           3,660
              Tanh-4                   [-1, 60]               0
            Linear-5                   [-1, 60]           3,660
              Tanh-6                   [-1, 60]               0
            Linear-7                   [-1, 60]           3,660
              Tanh-8                   [-1, 60]               0
            Linear-9                   [-1, 60]           3,660
             Tanh-10                   [-1, 60]               0
           Linear-11                    [-1, 1]              61
Total params: 14,881
Trainable params: 14,881
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [5]:
def train_classifier_one_epoch(train_data_loader, model, loss_function, optimizer,device):
    model.train()
    loss_individual_epoch = 0
    # for i, (train_input_x, train_input_t, train_output) in tqdm(enumerate(train_data_loader),'Training progress within an epoch'):
    for i, (train_input_x, train_input_t, train_output) in enumerate(train_data_loader):

        train_input_x, train_input_t, train_output = \
            train_input_x.to(device), train_input_t.to(device), train_output.to(device)

        optimizer.zero_grad(set_to_none=True)


        outputs = model(train_input_x, train_input_t)

        loss = loss_function(outputs, train_output.unsqueeze(1))
        loss.backward()
        optimizer.step()
        loss_individual_epoch += loss.item()

    loss_individual_epoch = loss_individual_epoch / len(train_data_loader)

    return loss_individual_epoch,model

def train_multiple_epochs(model, num_epochs, train_dataloader, loss_function, optimizer,device):
    tic = time.perf_counter()
    epoch = 1
    while epoch < num_epochs + 1:
        # print("Starting epoch number " + str(epoch))
        train_loss,model = train_classifier_one_epoch(train_dataloader, model, loss_function, optimizer,device)
        # print("Loss for Training on Epoch " +str(epoch) + " is "+ str(train_loss))
        if epoch % 200 == 0:
            toc = time.perf_counter()
            print(f'Epoch: {epoch}, Elapsed: {(toc-tic):.0f} sec')
            tic = time.perf_counter()
            print("Loss for Training on Epoch " +str(epoch) + " is "+ str(train_loss))

        epoch += 1

    return model

In [6]:
# my_mode_crack_classification=my_network_crack_prediction().to(device)
# loss_function=nn.BCELoss()
# learning_rate=0.001
# # optimizer=torch.optim.SGD(my_mode_crack_classification.parameters(), lr=learning_rate)
# optimizer=torch.optim.Adam(my_mode_crack_classification.parameters(), lr=learning_rate)
# test_frequency=2 #(how often do you want to display performance on the test data)


number_of_epochs=1e3#Choose the number of epochs
my_loss_func = nn.MSELoss()
my_optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

model=train_multiple_epochs(model,number_of_epochs,train_dataloader, my_loss_func, my_optimizer,device)

Epoch: 200, Elapsed: 101 sec
Loss for Training on Epoch 200 is 2.569498676330273e-10
Epoch: 400, Elapsed: 98 sec
Loss for Training on Epoch 400 is 3.112264543303489e-10
Epoch: 600, Elapsed: 102 sec
Loss for Training on Epoch 600 is 1.8551716648995686e-07


KeyboardInterrupt: 

In [11]:
x = np.linspace(0, 10, int(1e4))
y = np.sin(x)
len(y)

10000

In [37]:
torch.tensor(x).to(device)

tensor([0.0000e+00, 1.0001e-03, 2.0002e-03,  ..., 9.9980e+00, 9.9990e+00,
        1.0000e+01], device='cuda:0')

In [10]:
x = np.linspace(0, 10, int(3 * np.power(2, 12))).astype('float32')
y = np.sin(x).astype('float32')

class MyDataset_simple(torch.utils.data.Dataset):
    def __init__(self, x, y, device):
        self.x = torch.tensor(x.reshape(-1,1)).to(device)
        self.y = torch.tensor(y.reshape(-1,1)).to(device)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.x[i], self.y[i]

dataset_simple = MyDataset_simple(x, y, device)
my_batch_size = int(np.power(2, 12))
train_dataloader=torch.utils.data.DataLoader(dataset_simple, batch_size=my_batch_size)

In [12]:
class MyBenchModelSimple(nn.Module):
    def __init__(self):
        super(MyBenchModelSimple, self).__init__()
        #You code after this line

        self.my_layer = nn.Sequential(nn.Linear(1, 120),
                                 nn.Tanh(),
                                 nn.Linear(120, 120),
                                 nn.Tanh(),
                                 nn.Linear(120, 120),
                                 nn.Tanh(),
                                 nn.Linear(120, 120),
                                 nn.Tanh(),
                                 nn.Linear(120, 120),
                                 nn.Tanh(),
                                 nn.Linear(120, 1))

    def forward(self, x):
        #You code after this line
        x = self.my_layer(x)
        return x

model = MyBenchModelSimple().to(device)

In [14]:
summary(model, input_size=(1,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 120]             240
              Tanh-2                  [-1, 120]               0
            Linear-3                  [-1, 120]          14,520
              Tanh-4                  [-1, 120]               0
            Linear-5                  [-1, 120]          14,520
              Tanh-6                  [-1, 120]               0
            Linear-7                  [-1, 120]          14,520
              Tanh-8                  [-1, 120]               0
            Linear-9                  [-1, 120]          14,520
             Tanh-10                  [-1, 120]               0
           Linear-11                    [-1, 1]             121
Total params: 58,441
Trainable params: 58,441
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [15]:
epoch = 1
my_loss_func = nn.MSELoss()
my_optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

tic = time.perf_counter()
model.train()
while epoch < 1e3 + 1:
    loss_individual_epoch = 0
    for i, (train_input_x, train_input_y) in enumerate(train_dataloader):
        # my_optimizer.zero_grad(set_to_none=True)
        # my_optimizer.zero_grad()
        outputs = model(train_input_x)

        loss = my_loss_func(outputs, train_input_y)
        loss.backward()
        my_optimizer.step()
        loss_individual_epoch += loss.item()
    loss_individual_epoch = loss_individual_epoch / len(train_dataloader)

    if epoch % 200 == 0:
        toc = time.perf_counter()
        print(f'Epoch: {epoch}, Elapsed: {(toc-tic):.0f} sec')
        tic = time.perf_counter()
        print("Loss for Training on Epoch " +str(epoch) + " is "+ str(loss_individual_epoch))

    epoch += 1


Epoch: 200, Elapsed: 25 sec
Loss for Training on Epoch 200 is 0.6912877062956492


KeyboardInterrupt: 

In [9]:
epoch = 1
my_loss_func = nn.MSELoss()
my_optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

tic = time.perf_counter()
model.train()
while epoch < 1e3 + 1:
    loss_individual_epoch = 0

    my_optimizer.zero_grad()
    outputs = model(train_input_x)

    loss = my_loss_func(outputs, train_input_y)
    loss.backward()
    my_optimizer.step()
    loss_individual_epoch = loss.item()


    if epoch % 200 == 0:
        toc = time.perf_counter()
        print(f'Epoch: {epoch}, Elapsed: {(toc-tic):.2f} sec')
        tic = time.perf_counter()
        print("Loss for Training on Epoch " +str(epoch) + " is "+ str(loss_individual_epoch))

    epoch += 1

Epoch: 200, Elapsed: 2.20 sec
Loss for Training on Epoch 200 is 0.011960539035499096
Epoch: 400, Elapsed: 1.60 sec
Loss for Training on Epoch 400 is 0.009164966642856598
Epoch: 600, Elapsed: 1.65 sec
Loss for Training on Epoch 600 is 0.0063430555164813995
Epoch: 800, Elapsed: 1.68 sec
Loss for Training on Epoch 800 is 0.003994838800281286
Epoch: 1000, Elapsed: 1.69 sec
Loss for Training on Epoch 1000 is 0.002313939156010747


In [55]:
train_input_x.size()

torch.Size([1808, 1])

In [44]:
train_input_x.size()

torch.Size([1808])

In [45]:
train_input_x.unsqueeze(1).size()

torch.Size([1808, 1])

In [3]:
for i, (train_input_x, train_input_y) in enumerate(train_dataloader):
    print(i)

0


In [4]:
train_input_x.size()

torch.Size([12288, 1])

In [29]:
model(train_input_x.unsqueeze(1))

RuntimeError: expected scalar type Float but found Double

In [18]:
b.requires_grad

False

In [4]:
input_x = input_data['input'][:,0]
input_t = input_data['input'][:,1]
output_u = output_data['output']

NameError: name 'input_data' is not defined

In [41]:
type(output_u[3])

numpy.float64

In [13]:
len(input_data['input'])

32293

In [22]:
len(train_dataloader)

8

In [23]:
next(iter(train_dataloader))

[tensor([[8.5000e-02, 4.3200e-05],
         [0.0000e+00, 2.9920e-05],
         [8.5000e-02, 5.8640e-05],
         ...,
         [1.0000e-02, 4.5760e-05],
         [1.7000e-01, 4.4560e-05],
         [2.4500e-01, 4.9600e-05]], dtype=torch.float64),
 tensor([ 5.9278e-09, -7.6239e-09, -3.0900e-14,  ..., -2.1887e-13,
          7.3370e-09,  1.4075e-09], dtype=torch.float64)]

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class my_bench_model(nn.Module):
    def __init__(self):
        super(my_bench_model, self).__init__()
        #You code after this line

        self.my_layer = nn.Sequential(nn.Linear(2, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 60),
                                 nn.Tanh(),
                                 nn.Linear(60, 1))

    def forward(self, x):
        #You code after this line
        x = self.my_layer(x)
        return x

model = my_bench_model().to(device)

In [29]:
model(sample_data[0])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_mm)

In [45]:
a1.unsqueeze(0).size()

torch.Size([1, 2])

In [47]:
o1 = model(a1.unsqueeze(0))

In [48]:
o1.size()

torch.Size([1, 1])

In [42]:
a1.size()

torch.Size([2])

In [43]:
o1.size()

torch.Size([1])

In [33]:
a1[0]

tensor(0.0150, device='cuda:0', grad_fn=<SelectBackward0>)

In [38]:
(a1[0], a1[1])

(tensor(0.0150, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(0., device='cuda:0', grad_fn=<SelectBackward0>))

In [62]:
train_input

tensor([[8.5000e-02, 5.9280e-05],
        [2.3000e-01, 2.3280e-05],
        [2.2000e-01, 1.7520e-05],
        ...,
        [2.9500e-01, 3.4800e-05],
        [1.8000e-01, 3.2000e-07],
        [1.1000e-01, 1.5520e-05]])

In [63]:
train_input.size()

torch.Size([3621, 2])

In [64]:
train_input = train_input.to(device)


In [65]:
train_input.requires_grad = True

In [71]:
train_input.size()

torch.Size([3621, 2])

In [72]:
train_input[:,0]

tensor([0.0850, 0.2300, 0.2200,  ..., 0.2950, 0.1800, 0.1100], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [66]:
model_pred = model(train_input)
model_pred.size()

torch.Size([3621, 1])

In [77]:

summary(model, input_size=train_input.size())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 3621, 60]             180
              Tanh-2             [-1, 3621, 60]               0
            Linear-3             [-1, 3621, 60]           3,660
              Tanh-4             [-1, 3621, 60]               0
            Linear-5             [-1, 3621, 60]           3,660
              Tanh-6             [-1, 3621, 60]               0
            Linear-7             [-1, 3621, 60]           3,660
              Tanh-8             [-1, 3621, 60]               0
            Linear-9             [-1, 3621, 60]           3,660
             Tanh-10             [-1, 3621, 60]               0
           Linear-11              [-1, 3621, 1]              61
Total params: 14,881
Trainable params: 14,881
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/ba

In [73]:
derivs = dict()
# derivs['du_dx'], derivs['du_dt'] = torch.autograd.grad(model_pred, train_input, create_graph=True, grad_outputs=torch.ones_like(model_pred))
derivs['du_dx'] = torch.autograd.grad(model_pred, train_input[:,0], create_graph=True, grad_outputs=torch.ones_like(model_pred))

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [69]:
derivs

{'du_dx': (tensor([[ 0.0037, -0.0046],
          [ 0.0045, -0.0046],
          [ 0.0044, -0.0046],
          ...,
          [ 0.0049, -0.0046],
          [ 0.0042, -0.0046],
          [ 0.0038, -0.0046]], device='cuda:0', grad_fn=<MmBackward0>),)}

In [41]:
derivs = dict()
derivs['du_dx'], derivs['du_dt'] = torch.autograd.grad(o1, (a1[0], a1[1]), create_graph=True, grad_outputs=torch.ones_like(o1))

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [50]:
derivs = dict()
derivs['du_dx'], derivs['du_dt'] = torch.autograd.grad(o1, a1.unsqueeze(0), create_graph=True, grad_outputs=torch.ones_like(o1))

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [49]:
torch.ones_like(o1)

tensor([[1.]], device='cuda:0')

In [ ]:
derivs

In [35]:
torch.ones_like(a1)

tensor([1., 1.], device='cuda:0')

In [59]:
for i, (train_input, train_output) in tqdm(enumerate(train_dataloader)):
    print(train_output)
    print(train_output.size())


2it [00:00, 16.11it/s]

tensor([ 2.1526e-09, -2.1401e-08,  1.8544e-08,  ...,  1.4926e-09,
         1.5880e-09,  5.1902e-10])
torch.Size([4096])
tensor([0.0000e+00, 7.0251e-41, 5.2945e-20,  ..., 1.0940e-09, 3.7691e-12,
        4.8985e-33])
torch.Size([4096])
tensor([ 0.0000e+00,  6.9368e-10,  1.6562e-08,  ...,  6.7023e-14,
        -6.4639e-09,  4.2400e-09])
torch.Size([4096])
tensor([-7.9085e-32, -2.9370e-13,  1.5724e-25,  ...,  1.3347e-11,
         1.7906e-17,  2.7425e-13])
torch.Size([4096])
tensor([-7.3946e-09,  1.2916e-08,  2.7331e-13,  ..., -7.2592e-13,
         0.0000e+00,  1.3513e-08])
torch.Size([4096])


8it [00:00, 27.28it/s]

tensor([ 1.7250e-09, -2.6791e-08,  1.4163e-13,  ...,  1.9205e-09,
         3.9704e-14, -0.0000e+00])
torch.Size([4096])
tensor([ 1.9238e-13, -1.5105e-12,  8.2922e-09,  ..., -0.0000e+00,
         1.5730e-10,  0.0000e+00])
torch.Size([4096])
tensor([-1.1805e-13, -2.1199e-15, -6.2026e-20,  ..., -2.3504e-13,
        -0.0000e+00, -2.9698e-13])
torch.Size([3621])


In [61]:
train_input

tensor([[8.5000e-02, 5.9280e-05],
        [2.3000e-01, 2.3280e-05],
        [2.2000e-01, 1.7520e-05],
        ...,
        [2.9500e-01, 3.4800e-05],
        [1.8000e-01, 3.2000e-07],
        [1.1000e-01, 1.5520e-05]])

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [78]:
def train_classifier_one_epoch(train_data_loader, model, loss_function, optimizer,device):
    model.train()
    loss_individual_epoch = 0
    for i, (train_input, train_output) in tqdm(enumerate(train_data_loader),'Training progress within an epoch'):
        """
        Fill your code after this
        """
        train_input, train_output = train_input.to(device), train_output.to(device)
        # train_output = labels.to(torch.float32)

        optimizer.zero_grad(set_to_none=True)

        # images = images.to(device)
        # labels = labels.to(torch.short)
        outputs = model(train_input)

        loss = loss_function(outputs, train_output.unsqueeze(1))
        loss.backward()
        optimizer.step()
        loss_individual_epoch += loss.item()

    loss_individual_epoch = loss_individual_epoch / len(train_data_loader)

    return loss_individual_epoch,model

In [ ]:


aa, bb = train_classifier_one_epoch(train_dataloader)

In [84]:
my_loss_func = nn.MSELoss()
my_optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
aa = train_classifier_one_epoch(train_dataloader, model, my_loss_func, my_optimizer,device)

Training progress within an epoch: 8it [00:01,  7.54it/s]
